In [1]:
import numpy as np
import jax
import jax.numpy as jnp
import matplotlib.pyplot as plt
import scipy
from scipy.spatial import cKDTree
from scipy.spatial import Delaunay
from scipy.optimize import basinhopping
import copy
from sklearn.cluster import MiniBatchKMeans
from basic_functions import *
import time

2023-07-13 14:31:32.182904: W external/xla/xla/service/platform_util.cc:198] unable to create StreamExecutor for CUDA:0: failed initializing StreamExecutor for CUDA device ordinal 0: INTERNAL: failed call to cuDevicePrimaryCtxRetain: CUDA_ERROR_OUT_OF_MEMORY: out of memory; total memory reported: 12627279872
No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


In [2]:
#REAL trajectory
new_param = change_param(10000, "length")
ys_fixed = Euler_jnp(new_param)

subset_size = 500
#CELL CENTER
Xtrain_fixed = ys_fixed.transpose()
sample = MiniBatchKMeans(n_clusters=subset_size).fit(Xtrain_fixed).cluster_centers_

In [3]:
#point indices with respect to voronoi cells

def idxs(ys):
    point_idxs = []
    Xtrain = ys.transpose()#coordinates of points
    
    for i in range(new_param["length"]):
        
        distances = np.linalg.norm(sample - Xtrain[i], axis = 1)
        #Euclidean distance from the ith point to each cell
        
        idxs = np.argmin(distances)
        point_idxs.append(idxs)
    
    point_idxs = jnp.array(point_idxs)
    
    return point_idxs

In [4]:
#cell center of barycentric simplexes
vertices_cells = Delaunay(sample).simplices

#vertex coordinates of barycentric simplexes
vertices = sample[vertices_cells]

In [5]:
def precompute_barycentric_matrices(vertices):
    matrices = []
    for simplex in vertices:
        #ri: vertices of a Barycentric simplex
        r1, r2, r3, r4 = simplex

        x1, y1, z1 = r1
        x2, y2, z2 = r2
        x3, y3, z3 = r3
        x4, y4, z4 = r4
    
        T = np.array(([1., 1., 1., 1.],
                      [x1, x2, x3, x4],
                      [y1, y2, y3, y4],
                      [z1, z2, z3, z4]))
        T_inverse = np.linalg.inv(T)
        matrices.append(T_inverse)
    return matrices

In [6]:

#I use numpy here for now so this runs fast. But we probably need to jax when taking gradient.
#input:point coordinates
#output:(weights, indices)
def car2bar_weight(pt, barycentric_matrices):
                                 
    #Cartesian coordinates
    x, y, z = pt
    vec = np.array((1,x,y,z))#vector for barycentric matrix computing
    for i, mat in enumerate(barycentric_matrices):
        #ri: vertices of a Barycentric simplex
        bar = mat.dot(vec)

        if np.all(bar>= 0.):#to make sure the point is inside or on the simplex
            return bar, vertices_cells[i]
        
    distance = np.linalg.norm(sample - pt, axis = 1)
    index = np.argmin(distance)
    return np.array([1.]), np.array([index])
        


In [7]:
#generate idxs_tilda
def idxs_tilda(ys, barycentric_matrices):
    # start_time = time.time()

    point_idxs_tilda = np.zeros((subset_size, new_param["length"]))
    
    Xtrain = ys.transpose()
    
    for i in range(new_param["length"]):
        w, ind = car2bar_weight(Xtrain[i], barycentric_matrices)
        for num in range(len(w)):
            point_idxs_tilda[ind[num], i] += w[num]
        # if time.time() - start_time >= 10:
        #             print(f"i = {i}")
        #             start_time = time.time()
            
    return point_idxs_tilda

In [8]:
barycentric_matrices = precompute_barycentric_matrices(vertices)
point_idxs_tilda = idxs_tilda(ys_fixed, barycentric_matrices)

In [19]:
#Markov matrix constructed by making the (i, j) entry the sum of the products of the kth entry in the ith row and the (k+1)th entry in the jth row of point_idxs_tilda
#Dot product of point_idxs_tilda minus the last row and point_idxs_tilda minus the first row should be the same computation
markov_matrix = np.dot(point_idxs_tilda[:, :-1], point_idxs_tilda[:, 1:].T)
row_sums = np.sum(point_idxs_tilda, axis = 1)
markov_matrix = markov_matrix / row_sums[:, None]
%store markov_matrix


Stored 'markov_matrix' (ndarray)


In [10]:
#invariant measure computation
def power_method(mat, threshhold=1e-6, max_iterations=10000):
    x_prev = np.ones(subset_size)

    for _ in range(max_iterations):
        x = np.dot(mat, x_prev)
        x /= np.sum(x)

        # Check convergence
        if np.linalg.norm(x - x_prev) < threshhold:
            break
        x_prev = x.copy()
    x /= np.sum(x)
    return x

#teleportation regularization
def teleportation_regularization(mat, alpha = .85):
    ones = np.ones(subset_size)
    
    regularization_term = alpha * (1 / subset_size) * np.outer(ones, ones)
    mat_regularized = (1 - alpha) * mat + regularization_term
    
    return mat_regularized

#Power method + teleportation regularization
#have markov matrix. depending on quality of data, we can use different methods. 
#for example, if sparse data (need to figure out what sparse means here - what factors do we NEED and what can we go without) use only invariant measure, 
#otherwise can use multiple modes( 2nd, 3rd, etc eigenvector)

def find_invariant_measure(mat, alpha=1e-4, threshhold=1e-6, max_iterations=10000):
    mat_regularized = teleportation_regularization(mat, alpha)
    invariant_measure = power_method(mat_regularized, threshhold, max_iterations)
    return invariant_measure


In [11]:
old_markov_matrix = Markov_np(ys_fixed, sample)

In [12]:
inv = find_invariant_measure(markov_matrix)
old_inv = find_invariant_measure(old_markov_matrix)

In [13]:
#L2 distance of invariant measure
np.linalg.norm(inv - old_inv)

0.024200124600623492

In [14]:
#Frobenius distance
np.linalg.norm(markov_matrix - old_markov_matrix)

12.73362308691446